https://huggingface.co/learn/cookbook/rag_with_hugging_face_gemma_mongodb (ref)

In [2]:
!pip install -q datasets pandas pymongo sentence_transformers
!pip install -q -U transformers
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.7 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("AIatMongoDB/embedded_movies")
dataset_df = pd.DataFrame(dataset["train"])
dataset_df.head(2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

,cast,poster,title,runtime,imdb,rated,metacritic,awards,plot_embedding,countries,writers,directors,type,fullplot,genres,languages,num_mflix_comments,plot
0,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",https://m.media-amazon.com/images/M/MV5BMzgxOD...,The Perils of Pauline,199.0,"{'id': 4465, 'rating': 7.6, 'votes': 744}",None,NaN,"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[0.00072939653, -0.026834568, 0.013515796, -0....",[USA],"[Charles W. Goddard (screenplay), Basil Dickey...","[Louis J. Gasnier, Donald MacKenzie]",movie,Young Pauline is left a lot of money when her ...,[Action],[English],0,Young Pauline is left a lot of money when her ...
1,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",https://m.media-amazon.com/images/M/MV5BNzE1OW...,From Hand to Mouth,22.0,"{'id': 10146, 'rating': 7.0, 'votes': 639}",TV-G,NaN,"{'nominations': 1, 'text': '1 nomination.', 'w...","[-0.022837115, -0.022941574, 0.014937485, -0.0...",[USA],[H.M. Walker (titles)],"[Alfred J. Goulding, Hal Roach]",movie,As a penniless man worries about how he will m...,"[Comedy, Short, Action]",[English],0,A penniless young man tries to save an heiress...


In [4]:
dataset_df = dataset_df.dropna(subset=['fullplot'])
print(dataset_df.isnull().sum())

cast                    1
poster                 78
title                   0
runtime                14
imdb                    0
rated                 279
metacritic            893
awards                  0
plot_embedding          1
countries               0
writers                13
directors              12
type                    0
fullplot                0
genres                  0
languages               1
num_mflix_comments      0
plot                    0
dtype: int64


In [5]:
dataset_df = dataset_df.drop(columns=['plot_embedding'])

In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
embedding_model = SentenceTransformer("thenlper/gte-large")

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [8]:
def get_embedding(text):
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []
    embedding = embedding_model.encode(text)
    return embedding.tolist()

In [9]:
dataset_df["embedding"] = dataset_df["fullplot"].apply(get_embedding)

In [10]:
dataset_df.head(2)

,cast,poster,title,runtime,imdb,rated,metacritic,awards,countries,writers,directors,type,fullplot,genres,languages,num_mflix_comments,plot,embedding
0,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",https://m.media-amazon.com/images/M/MV5BMzgxOD...,The Perils of Pauline,199.0,"{'id': 4465, 'rating': 7.6, 'votes': 744}",None,NaN,"{'nominations': 0, 'text': '1 win.', 'wins': 1}",[USA],"[Charles W. Goddard (screenplay), Basil Dickey...","[Louis J. Gasnier, Donald MacKenzie]",movie,Young Pauline is left a lot of money when her ...,[Action],[English],0,Young Pauline is left a lot of money when her ...,"[-0.009285839274525642, -0.005062091629952192,..."
1,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",https://m.media-amazon.com/images/M/MV5BNzE1OW...,From Hand to Mouth,22.0,"{'id': 10146, 'rating': 7.0, 'votes': 639}",TV-G,NaN,"{'nominations': 1, 'text': '1 nomination.', 'w...",[USA],[H.M. Walker (titles)],"[Alfred J. Goulding, Hal Roach]",movie,As a penniless man worries about how he will m...,"[Comedy, Short, Action]",[English],0,A penniless young man tries to save an heiress...,"[-0.002439370146021247, 0.023095937445759773, ..."


In [11]:
import pymongo
from google.colab import userdata


def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None


mongo_uri = userdata.get("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client["movies"]
collection = db["movie_collection_2"]

Connection to MongoDB successful


In [12]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000002'), 'opTime': {'ts': Timestamp(1711290368, 1), 't': 2}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1711290368, 1), 'signature': {'hash': b'\x81<\x83\xfe\x0c\x9d\x0c\x9b\x93f\x7f\xb9RJ\x8ch>\xe8\xa2\x93', 'keyId': 7349464207743516677}}, 'operationTime': Timestamp(1711290368, 1)}, acknowledged=True)

In [13]:
documents = dataset_df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


In [14]:
def vector_search(user_query, collection):
    query_embedding = get_embedding(user_query)
    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,
                "limit": 4,
            }
        },
        {
            "$project": {
                "_id": 0,
                "fullplot": 1,
                "title": 1,
                "genres": 1,
                "score": {"$meta": "vectorSearchScore"},
            }
        },
    ]
    results = collection.aggregate(pipeline)
    return list(results)

In [15]:
def get_search_result(query, collection):
    get_knowledge = vector_search(query, collection)
    search_result = ""
    for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('fullplot', 'N/A')}\n"
    return search_result

In [16]:
query = "What is the best romantic movie to watch and why?"
source_information = get_search_result(query, collection)
combined_information = (
    f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}."
)

print(combined_information)

Query: What is the best romantic movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Shut Up and Kiss Me!, Plot: Ryan and Pete are 27-year old best friends in Miami, born on the same day and each searching for the perfect woman. Ryan is a rookie stockbroker living with his psychic Mom. Pete is a slick surfer dude yet to find commitment. Each meets the women of their dreams on the same day. Ryan knocks heads in an elevator with the gorgeous Jessica, passing out before getting her number. Pete falls for the insatiable Tiara, but Tiara's uncle is mob boss Vincent Bublione, charged with her protection. This high-energy romantic comedy asks to what extent will you go for true love?
Title: Pearl Harbor, Plot: Pearl Harbor is a classic tale of romance set during a war that complicates everything. It all starts when childhood friends Rafe and Danny become Army Air Corps pilots and meet Evelyn, a Navy nurse. Rafe falls head over heels and next thing you know

In [21]:
# import huggingface_hub
# huggingface_hub.login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# CPU Enabled uncomment below 👇🏽
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

In [22]:
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=500)
print(tokenizer.decode(response[0]))

<bos>Query: What is the best romantic movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Shut Up and Kiss Me!, Plot: Ryan and Pete are 27-year old best friends in Miami, born on the same day and each searching for the perfect woman. Ryan is a rookie stockbroker living with his psychic Mom. Pete is a slick surfer dude yet to find commitment. Each meets the women of their dreams on the same day. Ryan knocks heads in an elevator with the gorgeous Jessica, passing out before getting her number. Pete falls for the insatiable Tiara, but Tiara's uncle is mob boss Vincent Bublione, charged with her protection. This high-energy romantic comedy asks to what extent will you go for true love?
Title: Pearl Harbor, Plot: Pearl Harbor is a classic tale of romance set during a war that complicates everything. It all starts when childhood friends Rafe and Danny become Army Air Corps pilots and meet Evelyn, a Navy nurse. Rafe falls head over heels and next thing you

In [24]:
def search(query, model):
  source_information = get_search_result(query, collection)
  combined_information = (
    f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}.")
  input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
  response = model.generate(**input_ids, max_new_tokens=500)
  print(tokenizer.decode(response[0]))

In [27]:
query = "What is the best sci-fi and adventure movie to watch and why?"
search(query,model)

<bos>Query: What is the best sci-fi and adventure movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Space Raiders, Plot: A futuristic, sensitive tale of adventure and confrontation when a 10 year old boy is accidentally kidnapped by a spaceship filled with a motley crew of space pirates.
Title: Red Planet, Plot: In the near future, Earth is dying. A new colony on Mars could be humanity's only hope. A team of American astronauts, each a specialist in a different field, is making the first manned expedition to the red planet and must struggle to overcome the differences in their personalities, backgrounds and ideologies for the overall good of the mission. When their equipment suffers life-threatening damage and the crew must depend on one another for survival on the hostile surface of Mars, their doubts, fears and questions about God, man's destiny and the nature of the universe become defining elements in their fates. In this alien environment the